In [2]:
from PIL import Image
import numpy as np
import math
import os


num_class = 37
IMAGE_FORMAT = '.png'
INFER_DIR = './second_dataset/pred_new/im2_rgb/'
LABEL_DIR = './second_dataset/test/label2/'

In [24]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)


def get_hist(image, label):
    hist = np.zeros((num_class, num_class))
    hist += fast_hist(image.flatten(), label.flatten(), num_class)
    return hist


def cal_kappa(hist):
    if hist.sum() == 0:
        po = 0
        pe = 1
        kappa = 0
    else:
        print("here")
        po = np.diag(hist).sum() / hist.sum()
        pe = np.matmul(hist.sum(1), hist.sum(0).T) / hist.sum() ** 2
        if pe == 1:
            kappa = 0
        else:
            kappa = (po - pe) / (1 - pe)
    return kappa


def Eval():
    name_list = sorted(os.listdir(INFER_DIR))
    hist = np.zeros((num_class, num_class))
    for idx in range(len(name_list)):
        name = name_list[idx].split('.')[0]
        infer_file = INFER_DIR + '/' + str(name) + IMAGE_FORMAT
        label_file = LABEL_DIR + '/' + str(name) + IMAGE_FORMAT
        infer = Image.open(infer_file)
        label = Image.open(label_file)
        infer_array = np.array(infer)
        label_array = np.array(label)
        hist += get_hist(infer_array, label_array)

    print("Label distribution:", np.unique(label_array, return_counts=True))

    hist_fg = hist[1:, 1:]
    c2hist = np.zeros((2, 2))
    c2hist[0][0] = hist[0][0]
    c2hist[0][1] = hist.sum(1)[0] - hist[0][0]
    c2hist[1][0] = hist.sum(0)[0] - hist[0][0]
    c2hist[1][1] = hist_fg.sum()
    hist_n0 = hist.copy()
    hist_n0[0][0] = 0
    
    kappa_n0 = cal_kappa(hist_n0)
    iu = np.diag(c2hist) / (c2hist.sum(1) + c2hist.sum(0) - np.diag(c2hist))
    IoU_fg = iu[1]
    IoU_mean = (iu[0] + iu[1]) / 2
    print('Kappa = %.5f' % kappa_n0)
    print('IoU = %.5f' % IoU_fg)
    Sek = (kappa_n0 * math.exp(IoU_fg)) / math.e

    print('Mean IoU = %.5f' % IoU_mean)
    print('Sek = %.5f' % Sek)

In [25]:
Eval()

Label distribution: (array([  0, 128, 255], dtype=uint8), array([167644, 201896, 416892]))
here
Kappa = 0.00000
IoU = 1.00000
Mean IoU = 1.00000
Sek = 0.00000


In [26]:
infer = Image.open('./second_dataset/pred_new/im2_rgb/02161.png')

#show the image
infer.show()

In [31]:
import numpy as np

infer = Image.open('./second_dataset/pred_new/im2_rgb/02161.png')
infer_array = np.array(infer)

def rgb_to_class(image, colormap):
    # Convert colormap to a NumPy array for efficient processing
    colormap = np.array(colormap)
    class_map = np.zeros((image.shape[0], image.shape[1]), dtype=int)

    # Iterate over the colormap and assign class labels
    for class_idx, color in enumerate(colormap):
        matches = np.all(image == color, axis=-1)
        class_map[matches] = class_idx

    return class_map

# Example usage
ST_COLORMAP = [[255, 255, 255], [0, 0, 255], [128, 128, 128], [0, 128, 0], [0, 255, 0], [128, 0, 0], [255, 0, 0]]
class_array = rgb_to_class(infer_array, ST_COLORMAP)
print(class_array)

[[0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [30]:
infer_array = np.array(infer)
print(infer_array[0][511][0])
print(infer_array[0][511][1])
print(infer_array[0][511][2])

0
255
0


In [33]:
name_list = sorted(os.listdir(INFER_DIR))
hist = np.zeros((num_class, num_class))
infer = Image.open('./second_dataset/pred_new/im2_rgb/02161.png')
label = Image.open('./second_dataset/test/label2/02161.png')
infer_array = np.array(infer)
label_array = np.array(label)
infer_array = rgb_to_class(infer_array, ST_COLORMAP)
label_array = rgb_to_class(label_array, ST_COLORMAP)
hist += get_hist(infer_array, label_array)



hist_fg = hist[1:, 1:]
c2hist = np.zeros((2, 2))
c2hist[0][0] = hist[0][0]
c2hist[0][1] = hist.sum(1)[0] - hist[0][0]
c2hist[1][0] = hist.sum(0)[0] - hist[0][0]
c2hist[1][1] = hist_fg.sum()
hist_n0 = hist.copy()
hist_n0[0][0] = 0

kappa_n0 = cal_kappa(hist_n0)
iu = np.diag(c2hist) / (c2hist.sum(1) + c2hist.sum(0) - np.diag(c2hist))
IoU_fg = iu[1]
IoU_mean = (iu[0] + iu[1]) / 2
print('Kappa = %.5f' % kappa_n0)
print('IoU = %.5f' % IoU_fg)
Sek = (kappa_n0 * math.exp(IoU_fg)) / math.e

print('Mean IoU = %.5f' % IoU_mean)
print('Sek = %.5f' % Sek)

here
Kappa = 0.23811
IoU = 0.40299
Mean IoU = 0.62667
Sek = 0.13107


In [18]:
kappa_n0 = cal_kappa(hist_n0)


here
0.0 ////// 0.0


In [16]:
kappa_n0

0.0

In [13]:
hist_n0.sum()

32471139.0

In [4]:
Eval()

Kappa = 0.00000
IoU = 1.00000
Mean IoU = 1.00000
Sek = 0.00000


In [ ]:
"""Metrics for segmentation.
"""

import torch
import math
import numpy as np


class Metrics:
    """Tracking mean metrics
    """

    def __init__(self, labels):
        """Creates an new `Metrics` instance.

        Args:
          labels: the labels for all classes.
        """

        self.labels = labels

        self.tn = 0
        self.fn = 0
        self.fp = 0
        self.tp = 0

    def add(self, actual, predicted):
        """Adds an observation to the tracker.

        Args:
          actual: the ground truth labels.
          predicted: the predicted labels.
        """
        masks = torch.argmax(predicted, 0)
        # confusion = masks.view(-1).float() / actual.view(-1).float()
        ground_truths = actual
        pred = masks
        self.tn += np.array(torch.sum((ground_truths == 0) & (pred == 0)).cpu())
        self.fn += np.array(torch.sum((ground_truths == 1) & (pred == 0)).cpu())
        self.fp += np.array(torch.sum((ground_truths == 0) & (pred == 1)).cpu())
        self.tp += np.array(torch.sum((ground_truths == 1) & (pred == 1)).cpu())
        # a = self.tp / (self.tp + self.fp)
        # a = pred

        # self.tn += torch.sum(torch.isnan(confusion)).item()
        # self.fn += torch.sum(confusion == float("inf")).item()
        # self.fp += torch.sum(confusion == 0).item()
        # self.tp += torch.sum(confusion == 1).item()

    def get_precision(self):

        return self.tp / (self.tp + self.fp)

    def get_recall(self):

        return self.tp / (self.tp + self.fn)

    def get_f_score(self):

        pr = 2 *(self.tp / (self.tp + self.fp)) * (self.tp / (self.tp + self.fn))
        p_r = (self.tp / (self.tp + self.fp)) + (self.tp / (self.tp + self.fn))
        return pr / p_r

    def get_oa(self):
        
        t_pn = self.tp + self.tn
        t_tpn = self.tp + self.tn + self.fp + self.fn
        return t_pn / t_tpn

    def kappa(self):

        t_pn = self.tp + self.tn
        t_tpn = self.tp + self.tn + self.fp + self.fn
        po = t_pn / t_tpn
        pe_1 = (self.tn + self.fn)*(self.tn + self.fp) + (self.fp + self.tp)*(self.fn + self.tp)
        pe_2 = t_tpn * t_tpn
        pe = pe_1 / pe_2
        ka = (po - pe) / (1 - pe)
        return ka

    def get_miou(self):
        """Retrieves the mean Intersection over Union score.

        Returns:
          The mean Intersection over Union score for all observations seen so far.
        """
        return np.nanmean([self.tn / (self.tn + self.fn + self.fp), self.tp / (self.tp + self.fn + self.fp)])

    def get_fg_iou(self):
        """Retrieves the foreground Intersection over Union score.

        Returns:
          The foreground Intersection over Union score for all observations seen so far.
        """

        try:
            iou = self.tp / (self.tp + self.fn + self.fp)
        except ZeroDivisionError:
            iou = float("Inf")

        return iou

    def get_mcc(self):
        """Retrieves the Matthew's Coefficient Correlation score.

        Returns:
          The Matthew's Coefficient Correlation score for all observations seen so far.
        """

        try:
            mcc = (self.tp * self.tn - self.fp * self.fn) / math.sqrt(
                (self.tp + self.fp) * (self.tp + self.fn) * (self.tn + self.fp) * (self.tn + self.fn)
            )
        except ZeroDivisionError:
            mcc = float("Inf")

        return mcc


# Todo:
# - Rewrite mIoU to handle N classes (and not only binary SemSeg)